# Visualization Project

---
**Authors**:

- Juan Pablo Zaldivar
- Enric Millán
---

## Introduction

In this second project, the focus is on analyzing collision data in New York City during the summer of 2018. The primary objective is to develop a comprehensive interactive visualization that can address several key questions regarding the nature and patterns of these collisions. With the use of datasets related to collisions, weather conditions, and the New York City map, we aim to explore various facets, including ...

This file contains all the steps required to ensure reproducibility of steps leading from raw data to a clean dataset. The project is divided in three parts: the first part corresponds to the preprocessing of the data, the second part corresponds to the visualization desing process and the third part corresponds to the implementation of the visualization in the streamlit app to answer the questions.

The datasets are as follows:

- Collision Dataset: Extracting and filtering collision data specifically from June to September of 2018. This involves selecting relevant columns, handling missing or inconsistent data, and ensuring data quality.

- Weather Dataset: Locating and incorporating weather data corresponding to the time frames and areas of interest.

- New York City Map: Acquiring a suitable map of New York City to overlay geographical information related to collision locations.

### Dataset obtention and description

The Collisions dataset (`collisions_2018-2020.csv`) was extracted from the former project. The Motor Vehicle Collisions crash table contains details on the crash event. Each row represents a crash event. The Motor Vehicle Collisions data tables contain information from all police reported motor vehicle collisions in NYC. The dataset has to be preprocessed again in order to meet the new specifications.

The weather dataset (`weather2018.csv`) was already given by the supervisors of the project. It contains the weather conditions of the city of New York during the summer of 2018.

The map dataset was obtained from this [cartography web page](https://cartographyvectors.com/map/508-new-york-city-boroughs-ny).

The datasets are located in the folder `Data/`. Following are the loading of each dataset and the import of the required libraries.

### Libraries

For the correct functionality of the executions, the following folders and all their files are needed:

- `Data/`: Folder containing the datasets.
- `Modules/`: Folder containing the modules used in the project.

In [ ]:
# pip install altair==5.1.2 pandas==1.5.3 numpy==1.23.5 altair==5.1.2 h3pandas==0.2.5 geopandas==0.13.2 vegafusion[embed]>=1.4.0

: 

In [ ]:
import os
import time
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point 
from Modules import preprocessing as pre

: 

In [ ]:
dir = './Data'

: 

## Dataset Preprocessing

The preprocessing of the files involved a collaborative effort using both OpenRefine and selected Python libraries. This strategic approach was adopted to take advantage of the unique strengths and capabilities offered by each tool. OpenRefine facilitated initial data cleaning and transformation tasks, with its user-friendly interface for effective manipulation of datasets. Simultaneously, Python libraries were utilized to perform more complex data operations and manipulations, with special emphasis on the extensive functionalities and flexibility they provide. This combination allowed for a comprehensive preprocessing workflow that maximized efficiency and accuracy in preparing the data for subsequent analyses and visualization tasks.

### Collision Dataset Preprocessing

The initial step involved loading the original dataset into a `Pandas` dataframe, primarily to apply a date range filter efficiently. The rationale behind this approach was to optimize the data filtering process, considering the considerable size of the original dataset. The volume of data posed challenges within OpenRefine, leading to slow and inefficient computational processes. By filtering the dataset using `Pandas`, it allowed for a more streamlined and quicker selection of the desired date range. Following this initial filtering phase, the refined dataset was exported as a `.csv` file (`collisions-2018.csv`) and subsequently imported into OpenRefine for further data processing and cleaning procedures. This sequential approach ensured a balance between computational efficiency and data handling capabilities across both Pandas and OpenRefine, resulting in a more effective preprocessing workflow.

In [ ]:
collisions = pd.read_csv(dir + '/collisions_2018-2020.csv')
collisions.head(1)

: 

In [ ]:
collisions['YEAR'] = pd.DatetimeIndex(collisions['CRASH DATE']).year
collisions.shape

: 

In [ ]:
collisions = collisions[collisions['YEAR'] == 2018]
collisions.shape

: 

#### Data Type Conversion

In OpenRefine, the data conversion process involved several attribute adjustments. The CRASH DATE attribute underwent a conversion to a date type for enhanced consistency and data clarity. Meanwhile, both COLLISION ID and CRASH TIME were temporarily set as string types.

Attributes pertaining to the geographical location of the collisions were modified to strings, accompanied by specific notations. As part of this process, all values were standardized to uppercase, and any extra spaces were removed where applicable. This standardization was implemented to facilitate the effectiveness of the clustering method utilized for collectively inspecting and modifying cells. The objective was to streamline the identification and correction of any inconsistencies or inaccuracies within the data, ensuring a more uniform and reliable dataset for subsequent analyses.

The attributes pertaining to the number of persons involved in the collision underwent a data type conversion to integers within the dataset. This decision was driven by the discrete nature of these values and the expectation that these numerical counts wouldn't contain negative values.

Conversely, the attributes related to vehicles and factors involved in the collisions were retained as strings temporarily. This choice was made to maintain flexibility in handling these attributes during subsequent data processing and analysis phases, ensuring that any necessary modifications or categorizations could be applied effectively as the analysis progressed.

#### Data Selection and Transformation

From previous knowledge of the dataset, a subset of attributes was selected for further analysis. This selection was based on the relevance of the attributes to the research questions and the availability of data.

In [ ]:
interest_cols = ['CRASH DATE', 'CRASH TIME', 'BOROUGH', 'ZIP CODE', 'LATITUDE', 'LONGITUDE', 'ON STREET NAME', 'OFF STREET NAME', 'NUMBER OF PERSONS INJURED', 'NUMBER OF PERSONS KILLED', 'NUMBER OF PEDESTRIANS INJURED', 'NUMBER OF PEDESTRIANS KILLED', 'NUMBER OF CYCLIST INJURED', 'NUMBER OF CYCLIST KILLED', 'NUMBER OF MOTORIST INJURED', 'NUMBER OF MOTORIST KILLED', 'CONTRIBUTING FACTOR VEHICLE 1', 'COLLISION_ID', 'VEHICLE TYPE CODE 1']

collisions = collisions[interest_cols]

: 

**Geographic attributes**

As we saw in the previous project, **ON STREET NAME** and **OFF STREET NAME** seem to be the same attribute, but with different names. The web site of the dataset cointains the following descriptions:

- **ON STREET NAME**: *Street on which the collision occurred*.
- **OFF STREET NAME**: *Street address if known*.

This gives the idea that both attributes probably contain the same information. Furthermore, there are no rows with both attributes filled, which makes the idea of merging both attributes plausible and would consolidate information without redundancy.

In [ ]:
collisions[(collisions['ON STREET NAME'].notnull()) & (collisions['OFF STREET NAME'].notnull())].shape

: 

In [ ]:
collisions[(collisions['ON STREET NAME'].notnull()) | (collisions['OFF STREET NAME'].notnull())].shape

: 

The resulting attribute after merging both columns is called **STREET NAME** and contains the street name/address where the collision occurred, with no missing values. Some rows will have a more detailed description of the street, while others will only have the name of the street. The fusion of the two columns will be done in OpenRefine.

The clustering process utilized the key collision method in conjunction with the fingerprint keying function, applied separately to each individual geographical attribute. After a couple of iterations, no substantial alterations in attribute values were identified. However, during this process, misspellings were detected and rectified to ensure data accuracy.

The misspellings were corrected and the clusterization was done again. The results were the same, which means that the values were already consistent. To verify the result, a Neares Neighbours analysis was done as well but without finding any significant variation.

**Vehicle attributes**

We have seen already that there are many classes of the VEHICLE CODE TYPE 1 values. To simplify this complexity, the project statement force to reduce the diversity of classes by employing a clustering technique. This clustering methodology allowed us to condense the multitude of classes within VEHICLE CODE TYPE 1 into a more manageable set of clusters, facilitating a more comprehensible and concise representation for subsequent visualization and analysis purposes.

In [ ]:
# PASAR ESTA FUNC A PREPROCESSING

collisions['VEHICLE TYPE CODE 1'] = collisions['VEHICLE TYPE CODE 1'].replace(['Taxi', 'TAXI'], 'TAXI')
collisions['VEHICLE TYPE CODE 1'] = collisions['VEHICLE TYPE CODE 1'].replace(['Fire', 'FD tr', 'firet', 'fire', 'FIRE', 'fd tr', 'FD TR', 'FIRE', 'FIRET'], 'FIRE')
collisions['VEHICLE TYPE CODE 1'] = collisions['VEHICLE TYPE CODE 1'].replace(['AMBUL', 'Ambulance', 'ambul', 'AMB', 'Ambul', 'AMBULANCE', 'AMBU'], 'AMBULANCE')

: 

The resulting classes are the following:

- **TAXI**: 
- **FIRE**:
- **AMBULANCE**:

A similar strategy was done with the CONTRIBUTING FACTOR VEHICLE 1 attribute. However, the aggregation was not so exhaustive since this attribute wasn't needed a priori for the main questions that the visualizations should answer. For this attribute basic merge transformations were applied in OpenRefine until no "strange" or "uninformative" nor repeated classes remained.

**Number of persons attributes**

For the visualization purposes, the differentantion of **PERSONS**, **PEDESTRIANS**, **CYCLISTS** and **MOTORISTS** (**INJURED/KILLED**) is irrelevant. A more useful attribute would be the total number of persons involved in the collision. This can be obtained by summing the four attributes under the assumption that the **PERSONS** attribute is not the sum of the other three attributes.

This condition was needed to be checked because the documentation of the dataset was not precise enough to determinate if **NUMBER OF PERSON INJURED/KILLED** was an aggregate from the other three columns or not.

*Note: The metadata information available in the web of the dataset was: "Number of persons injured/killed" regarding the **NUMBER OF PERSONS INJURED/KILLED**.*

In [ ]:
collisions['NUMBER OF PERSONS INJURED'].equals(collisions['NUMBER OF PEDESTRIANS INJURED'] + collisions['NUMBER OF CYCLIST INJURED'] + collisions['NUMBER OF MOTORIST INJURED'])

: 

In [ ]:
collisions['NUMBER OF PERSONS INJURED'].equals(collisions['NUMBER OF PEDESTRIANS INJURED'])

: 

In [ ]:
collisions['NUMBER OF PERSONS KILLED'].equals(collisions['NUMBER OF PEDESTRIANS KILLED'] + collisions['NUMBER OF CYCLIST KILLED'] + collisions['NUMBER OF MOTORIST KILLED'])

: 

In [ ]:
collisions['NUMBER OF PERSONS KILLED'].equals(collisions['NUMBER OF PEDESTRIANS KILLED'])

: 

As seen by the logical comprobations, the **NUMBER OF PERSONS INJURED/KILLED** is not the sum of the other three attributes. Furthermore, the terms persons and pedestrians are not equal, as one could have thought that the term persons was used to refer to pedestrians.

Based on this, the discrete attributes refering to the injured people were summed to obtain **NUMBER OF INJURED** and the discrete attributes refering to the killed people were summed to obtain **NUMBER OF KILLED**. The **NUMBER OF INJURED/KILLED** attributes were removed.

In [ ]:
# PASAR ESTA FUNC A PREPROCESAMIENTO

collisions['TOTAL INJURED'] = collisions.filter(regex='INJURED').sum(axis=1)
collisions['TOTAL KILLED'] = collisions.filter(regex='KILLED').sum(axis=1)

collisions = collisions.drop(collisions.iloc[:, 8:16], axis=1)

: 

---

In [ ]:
collisions.to_csv(dir + '/collisions-2018.csv', index=False)

: 

In [ ]:
collisions = pd.read_csv(dir + '/collisions-2018_prepro_v1.csv')

: 

ESTO SE HACE PORQ EL PROFE SE EQUIVOCO

In [ ]:
collisions = collisions[collisions['VEHICLE TYPE CODE 1'].isin(['FIRE', 'TAXI', 'AMBULANCE'])]

collisions['VEHICLE TYPE CODE 1'].unique()

: 

In [ ]:
collisions.shape

: 

At this point, the dataset contains the attributes needed (except for weather attributes) for the analysis and some extra attributes that were considered interesting for some possible extra analysis or insights that we could be added to the visualization.

---

#### Missing Values

It has already been mentioned the existence of some missing values. In the previous section, the verification of missing values was done with the .isnull() method of Pandas. However, this method does not take into account the `NaN` values. In order to check the existence of NaN values, the .isna() method was used.

In [ ]:
comp = (collisions.isnull().sum() == collisions.isna().sum())
comp[comp == False]

: 

As seen previously, all the missing values of the dataset are detected both with .isnull() and .isna(). After this check, notice that the only attributes with missign values are corresponding to geographical properties of the collisions. We can apply a similar strategy as the one used in the previous project to fill the missing values.

In [ ]:
collisions.isnull().sum()

: 

In [ ]:
collisions['STREET NAME'] = collisions['STREET NAME'].apply(pre.capitalize_street)

: 

Initially, our focus lies in addressing the missing attributes within the coordinates. Our approach involves leveraging the information present in the **STREET NAME** column, which is devoid of any missing values. The outlined process is as follows:

Utilization of the `Nominatim` geocoding service enables us to retrieve coordinates corresponding to the street names. Whenever available, we employ the BOROUGH or ZIP CODES to refine our search parameters. Upon a successful search, we populate the missing values with the obtained coordinates. However, in cases where the search yields no results, we maintain the missing values in their current state. This is done by the `pre.fill_missing_coordinates()` function.

In [ ]:
if not os.path.exists(dir + '/collisions-2018_prepro_v2.csv'):
    collisions = collisions.apply(pre.fill_missing_coordinates, axis=1)
    collisions.to_csv(dir + '/collisions-2018_prepro_v2.csv', index=False)
else:
    collisions = pd.read_csv(dir + '/collisions-2018_prepro_v2.csv')

: 

In [ ]:
collisions.isnull().sum()

: 

In [ ]:
34*100/collisions.shape[0]

: 

The proportion of missing values reduces notably for the coordinate values, accounting for only $0.83%$ of the total rows. Addressing the missing entries in the **BOROUGH** and **ZIP CODE** columns involves a strategy that involves verifying whether a given point, defined by its coordinates (**LONGITUDE**, **LATITUDE**), resides within the boundary polygon of distinct **BOROUGH** or **ZIP CODE** values. When the original attribute value is null, this methodology endeavors to allocate the appropriate borough based on the geographic coordinates provided.

For that, the extraction of polygons for both the **BOROUGH** and **ZIP CODE** attributes is imperative. The polygons for the **BOROUGH** attribute were acquired from the existing files accessible within the `./Data` folder.

In [ ]:
borough_poly = pre.get_borough_polygons()

: 

In [ ]:
zip_poly = pre.get_zip_polygons()

: 

In [ ]:
pre.fill_missing_borough_zip(collisions, borough_poly, zip_poly)

: 

In [ ]:
collisions['BOROUGH'].value_counts()

: 

From this breakdown, it's evident that Manhattan has the highest count of imputed values, followed by Brooklyn, Queens, Bronx, and finally Staten Island with the lowest count. This distribution suggests that a significant portion of missing values were inferred and filled with these borough names. The disparities in counts might be attributed to various factors such as the availability of geographic data or the frequency of missing values per borough.

In [ ]:
collisions.isnull().sum()

: 

A noticeable reduction in the count of missing values is apparent in the **BOROUGH** and **ZIP CODE** attributes compared to the missing values found in the coordinates. The relatively diminished count of missing values in these attributes allows for their removal from the dataset without impacting the analytical outcomes. This removal is feasible because these missing values represent a small proportion and their exclusion does not significantly affect the analysis.

In [ ]:
collisions.isnull().any(axis=1).sum()

: 

In [ ]:
collisions = collisions.dropna()
collisions['BOROUGH'] = collisions['BOROUGH'].apply(pre.capitalize_boroughs)

: 

Finally, we impute some extra variables that will be useful to have different aggregation levels for time in the visualization: **HOUR**, **MONTH** (with number), **WEEKDAY** and **MONTH** (with name).

In [ ]:

collisions['HOUR'] = pd.to_datetime(collisions['CRASH TIME']).dt.hour
collisions['MONTH'] = pd.to_datetime(collisions['CRASH DATE']).dt.month

collisions = collisions[collisions['LONGITUDE'] != 0]

collisions['WEEKDAY'] = pd.to_datetime(collisions['CRASH DATE']).dt.day_name()
collisions['MONTH'] = pd.to_datetime(collisions['CRASH DATE']).dt.month_name()

: 

In [ ]:
collisions.to_csv(f'{dir}/collisions_clean.csv', index=False)

: 

---

### Weather Dataset Preprocessing

In [ ]:
weather = pd.read_csv(f'{dir}/weather2018.csv')
weather.head(1)

: 

In [ ]:
weather.shape

: 

In [ ]:
weather.describe()

: 

In [ ]:
weather.isnull().sum()

: 

In [ ]:
weather['icon'].unique()

: 

For desing purposes, we decided to change the **icon** variable values to the following ones:

In [ ]:
weather['icon'] = weather['icon'].replace('rain', 'Rainy')
weather['icon'] = weather['icon'].replace('clear-day', 'Clear day')
weather['icon'] = weather['icon'].replace('partly-cloudy-day', 'Partly cloudy day')
weather['icon'] = weather['icon'].replace('cloudy', 'Cloudy')

: 

And now, we select the column that we need to answer the proposed questions, and some extra variables that may be useful for an extra visualization:

In [ ]:
weather = weather[['datetime', 'tempmax', 'tempmin', 'icon']]

# transform the nammes of the columns to uppercase

# weather.columns = weather.columns.str.upper()

weather.to_csv(f'{dir}/weather_clean.csv', index=False)

: 